In [1]:
# importing modules
import pandas as pd
import numpy as np
from IPython.display import display

# sql libraries
from sqlalchemy import create_engine
from sqlalchemy.types import DateTime, VARCHAR
import mysql.connector as c


In [2]:
# importing data
# VisitorLogsData
visitorlogs = pd.read_csv('VisitorLogsData.csv')
print('Shape of VisitorLogsData', visitorlogs.shape)
display(visitorlogs.head())

Shape of VisitorLogsData (6588000, 9)


,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
0,WI10000050298,2018-05-07 04:28:45.970,pr100631,NaN,NaN,Chrome Mobile,Android,Chennai,India
1,WI10000025922,2018-05-13 07:26:04.964,pr100707,NaN,NaN,Chrome,Windows,NaN,Taiwan
2,WI100000204522,2018-05-11 11:43:42.832,pr100030,NaN,click,Chrome,windows,Gurgaon,India
3,WI10000011974,2018-05-13 15:20:23.436,Pr100192,NaN,CLICK,Chrome,Windows,,
4,WI100000441953,2018-05-08 20:44:25.238,Pr100762,NaN,click,Chrome,mac os x,Iselin,United States


In [3]:
# userTable data
user = pd.read_csv('userTable.csv')
print('Shape of userTable', user.shape)
display(user.head())


Shape of userTable (34050, 3)


,UserID,Signup Date,User Segment
0,U133159,2018-04-14 07:01:16.202607+00:00,C
1,U129368,2017-12-02 09:38:41.584270+00:00,B
2,U109654,2013-03-19 11:38:55+00:00,B
3,U108998,2018-01-18 08:29:51.627954+00:00,C
4,U131393,2018-03-27 08:05:28.806800+00:00,B


In [4]:
# renaming user columns for ease of use
user.rename(columns={'Signup Date': 'SignupDate',
                     'User Segment': 'Segment'}, inplace=True)
user.head()

,UserID,SignupDate,Segment
0,U133159,2018-04-14 07:01:16.202607+00:00,C
1,U129368,2017-12-02 09:38:41.584270+00:00,B
2,U109654,2013-03-19 11:38:55+00:00,B
3,U108998,2018-01-18 08:29:51.627954+00:00,C
4,U131393,2018-03-27 08:05:28.806800+00:00,B


In [5]:
# checking for null values
visitorlogs.isnull().sum()


webClientID            0
VisitDateTime     658915
ProductID         527137
UserID           5937305
Activity          889446
Browser                0
OS                     0
City             2165831
Country           397693
dtype: int64

 We have 5937305 UserID null values.
 Since the output table requires data related to only users
 who have a UserID. Therefore we drop column that does not contain UserID

In [6]:
# dropping unregistered users
visitorlogs.dropna(subset=['UserID'], inplace=True)

# checking for null values
visitorlogs.isnull().sum()

webClientID           0
VisitDateTime     64803
ProductID         51963
UserID                0
Activity          87757
Browser               0
OS                    0
City             236124
Country           42827
dtype: int64

In [7]:
# checking record count
print('Filtered shape of data is', visitorlogs.shape)


Filtered shape of data is (650695, 9)


#### Formatting timestamp

In [8]:
# creating a function to convert all time formats into single datetime

def datetimecleaning(date_time):
    '''
    Converts the strings to datetime format.
    input=datetime string format
    output=datetime format
    '''
    

    try:
        return pd.to_datetime(pd.Timestamp(date_time), utc=True)
    except:
        try:
            date_time = int(date_time)
            return pd.to_datetime(pd.Timestamp(date_time), utc=True)
        except:
            pass


In [9]:
# applying datetime cleaning
visitorlogs['VisitDateTime'] = visitorlogs['VisitDateTime'].map(lambda x: datetimecleaning(x))
user['SignupDate'] = user['SignupDate'].map(lambda x: datetimecleaning(x))


In [10]:
# displaying cleaned data
display(visitorlogs.head())
print('--'*30)
display(user.head())


,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
14,WI100000112772,2018-05-15 06:48:15.249000+00:00,Pr100017,U106593,click,Chrome Mobile,Android,,
21,WI1000009977,2018-05-23 07:02:01.790000+00:00,Pr101008,U108297,NaN,Chrome Mobile,Android,Delhi,India
23,WI100000223826,2018-05-10 06:28:53.391000+00:00,Pr100241,U132443,NaN,Firefox,Windows,NaN,India
24,WI10000021998,2018-05-08 12:40:02.153000+00:00,pr100495,U134616,click,Chrome,Windows,Cottage Grove,United States
33,WI10000036281,2018-05-11 15:35:43.689000+00:00,Pr100363,U130784,click,Chrome,Chrome OS,New Delhi,India


------------------------------------------------------------


,UserID,SignupDate,Segment
0,U133159,2018-04-14 07:01:16.202607+00:00,C
1,U129368,2017-12-02 09:38:41.584270+00:00,B
2,U109654,2013-03-19 11:38:55+00:00,B
3,U108998,2018-01-18 08:29:51.627954+00:00,C
4,U131393,2018-03-27 08:05:28.806800+00:00,B


In [11]:
# replacing empty spaces with NaN
visitorlogs = visitorlogs.replace(r'^\s*$', np.nan, regex=True)
user = user.replace(r'^\s*$', np.nan, regex=True)


In [12]:
# checking for null values
visitorlogs.isnull().sum()


webClientID           0
VisitDateTime     64803
ProductID         51963
UserID                0
Activity          87757
Browser               0
OS                    0
City             253493
Country           60196
dtype: int64

 Since we have lot of null values in VisitDateTime column we impute it with average datetime for that respective user

In [13]:
# imputing NaN with datetime mean of each user
visitorlogs['VisitDateTime'] = visitorlogs.groupby(['UserID'])['VisitDateTime']\
    .transform(lambda x: x.fillna(x.mean()))


In [14]:
# checking for null values
visitorlogs.isnull().sum()


webClientID           0
VisitDateTime       805
ProductID         51963
UserID                0
Activity          87757
Browser               0
OS                    0
City             253493
Country           60196
dtype: int64

 this cleaned data will now be exported to sql for further processing

In [15]:
# creating sql connection
con = c.connect(host='localhost', user='root', passwd='235555')


In [16]:
# checking if connection is successful
if con.is_connected():
    print('Successfully connected')

Successfully connected


In [17]:
# creating a cursor to execute query
mycursor = con.cursor()


In [18]:
# dropping database if it exists
mycursor.execute("DROP DATABASE IF EXISTS comz_database_01")


In [19]:
# creating a database
mycursor.execute("CREATE DATABASE comz_database_01")

In [20]:
# checking if database is created
mycursor.execute("SHOW DATABASES")

# printing list of all databases
for x in mycursor:
    print(x)


('comz_database',)
('comz_database_01',)
('comz_db',)
('comz_db_processed',)
('information_schema',)
('mavenfuzzyfactory',)
('mysql',)
('performance_schema',)
('sys',)


In [21]:
engine = create_engine("mysql+mysqlconnector://" + 'root' + ":"
                       + '235555' + "@" + 'localhost' + "/" + 'comz_database_01')


In [22]:
# tables in database

query = 'Show tables from comz_database_01'
tables = pd.read_sql_query(query, con)
tables


,Tables_in_comz_database_01


 currently there are no tables

In [23]:
# creating variable dtypes for visitorlogdata
dtype = {'webClientID': VARCHAR(50),
         'VisitDateTime': DateTime,
         'ProductID': VARCHAR(50),
         'UserID': VARCHAR(50),
         'Activity': VARCHAR(50),
         'Browser': VARCHAR(50),
         'OS': VARCHAR(50),
         'City': VARCHAR(100),
         'Country': VARCHAR(100)}

In [24]:
# exporting data to sql
visitorlogs.to_sql(name='visitorlogsdata', con=engine,
                   if_exists='replace', index=False, dtype=dtype)

In [25]:
# creating variable dtypes for usertable
dtype = {'UserID': VARCHAR(10),
         'SignupDate': DateTime,
         'Segment': VARCHAR(5)}

In [26]:
# exporting data to sql
user.to_sql(name='usertable', con=engine, if_exists='replace', index=False)



In [27]:
# updating sql connection with created database
con = c.connect(host='localhost', user='root', passwd='235555',
                database='comz_database_01')


In [28]:
# checking if data is imported
query = 'Show tables from comz_database_01'
tables = pd.read_sql_query(query, con)
tables

,Tables_in_comz_database_01
0,usertable
1,visitorlogsdata


In [29]:
# printing visitorlogsdata
pd.read_sql_query('Select * from visitorlogsdata', con)


,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
0,WI100000112772,2018-05-15 06:48:15,Pr100017,U106593,click,Chrome Mobile,Android,None,None
1,WI1000009977,2018-05-23 07:02:02,Pr101008,U108297,None,Chrome Mobile,Android,Delhi,India
2,WI100000223826,2018-05-10 06:28:53,Pr100241,U132443,None,Firefox,Windows,None,India
3,WI10000021998,2018-05-08 12:40:02,pr100495,U134616,click,Chrome,Windows,Cottage Grove,United States
4,WI10000036281,2018-05-11 15:35:44,Pr100363,U130784,click,Chrome,Chrome OS,New Delhi,India
...,...,...,...,...,...,...,...,...,...
650690,WI100000933943,2018-05-11 15:51:44,None,U129703,pageload,Safari,Mac OS X,Cypress,United States
650691,WI1000003495,2018-05-21 05:20:36,pr101042,U104056,None,Chrome,Windows,Dhaka,Bangladesh
650692,WI100000360,2018-05-25 11:28:57,Pr100002,U106239,click,Chrome,Windows,None,India
650693,WI10000094643,2018-05-10 12:23:22,pr100060,U111562,click,Chrome,windows,Ulan Bator,Mongolia


In [30]:
# printing visitorlogsdata
pd.read_sql_query('Select * from usertable', con)


,UserID,SignupDate,Segment
0,U133159,2018-04-14 07:01:16,C
1,U129368,2017-12-02 09:38:42,B
2,U109654,2013-03-19 11:38:55,B
3,U108998,2018-01-18 08:29:52,C
4,U131393,2018-03-27 08:05:29,B
...,...,...,...
34045,U134073,2018-03-19 12:35:11,B
34046,U113667,2018-02-28 08:29:23,B
34047,U128470,2018-03-04 13:03:13,B
34048,U104005,2018-01-22 16:16:35,B


In [31]:
# setting variable max date
# setting we have to filter data from 27th july 2018 we set variable for ease of further use
query = "select @current_date:= STR_TO_DATE('2018-05-27 23:59:59','%Y-%m-%d %H:%i:%s.%f')"
max_date = pd.read_sql_query(query, con)
max_date


,"@current_date:= STR_TO_DATE('2018-05-27 23:59:59','%Y-%m-%d %H:%i:%s.%f')"
0,2018-05-27 23:59:59.000000


In [32]:
# creating results dataframe in which result of each query will be stores
results = pd.DataFrame()

In [33]:
# 1) Unique ID of the registered user

results['UserID'] = sorted(user.UserID)
results


,UserID
0,U100002
1,U100003
2,U100004
3,U100005
4,U100006
...,...
34045,U136960
34046,U136961
34047,U136963
34048,U136964


In [34]:
# 2) How many days a user was active on platform in the last 7 days.

query = 'select UserID, count(distinct day(VisitDateTime)) as No_of_days_Visited_7_Days\
        from visitorlogsdata\
        where VisitDateTime>@current_date-interval 7 day\
        and UserID is not null\
        group by UserID\
        order by UserID\
        ;'

# ececuting query
No_of_days_Visited_7_Days = pd.read_sql_query(query, con)
No_of_days_Visited_7_Days

,UserID,No_of_days_Visited_7_Days
0,U100003,1
1,U100004,1
2,U100005,1
3,U100006,1
4,U100008,7
...,...,...
16167,U119707,1
16168,U119708,1
16169,U119709,1
16170,U119711,1


In [35]:
# updating results
results['No_of_days_Visited_7_Days'] = pd.merge(results, No_of_days_Visited_7_Days,
                                                how='left', on='UserID')['No_of_days_Visited_7_Days']
results


,UserID,No_of_days_Visited_7_Days
0,U100002,NaN
1,U100003,1.0
2,U100004,1.0
3,U100005,1.0
4,U100006,1.0
...,...,...
34045,U136960,NaN
34046,U136961,NaN
34047,U136963,NaN
34048,U136964,NaN


In [36]:
# 3) Number of Products viewed by the user in the last 15 days

query = 'select UserID,count(distinct ProductID) as No_Of_Products_Viewed_15_Days\
        from visitorlogsdata\
        where VisitDateTime> @current_date -interval 15 day\
        and UserID is not null\
        group by UserID\
        order by UserID\
        ;'
# executing query
No_Of_Products_Viewed_15_Days = pd.read_sql_query(query, con)
No_Of_Products_Viewed_15_Days


,UserID,No_Of_Products_Viewed_15_Days
0,U100002,2
1,U100003,2
2,U100004,15
3,U100005,4
4,U100006,1
...,...,...
25880,U129368,1
25881,U129369,1
25882,U129370,1
25883,U129371,3


In [37]:
# updating results
results['No_Of_Products_Viewed_15_Days'] = pd.merge(results, No_Of_Products_Viewed_15_Days,
                                                    how='left', on='UserID')['No_Of_Products_Viewed_15_Days']
results


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days
0,U100002,NaN,2.0
1,U100003,1.0,2.0
2,U100004,1.0,15.0
3,U100005,1.0,4.0
4,U100006,1.0,1.0
...,...,...,...
34045,U136960,NaN,NaN
34046,U136961,NaN,NaN
34047,U136963,NaN,NaN
34048,U136964,NaN,NaN


In [38]:
# 4) Vintage (In Days) of the user as of today

query = 'select UserID,datediff(@current_date,SignupDate) as User_Vintage\
        from usertable\
        where UserID is not null\
        order by UserID\
        ;'

# executing query
User_Vintage = pd.read_sql_query(query, con)
User_Vintage


,UserID,User_Vintage
0,U100002,52
1,U100003,1020
2,U100004,340
3,U100005,680
4,U100006,54
...,...,...
34045,U136960,754
34046,U136961,59
34047,U136963,1419
34048,U136964,494


In [39]:
# updating results
results['User_Vintage'] = pd.merge(results, User_Vintage,
                                   how='left', on='UserID')['User_Vintage']
results


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage
0,U100002,NaN,2.0,52
1,U100003,1.0,2.0,1020
2,U100004,1.0,15.0,340
3,U100005,1.0,4.0,680
4,U100006,1.0,1.0,54
...,...,...,...,...
34045,U136960,NaN,NaN,754
34046,U136961,NaN,NaN,59
34047,U136963,NaN,NaN,1419
34048,U136964,NaN,NaN,494


In [40]:
# 5) Most_Viewed_product_15_Days

query = 'select UserID,Most_Viewed_product_15_Days ,max(occurs) as products_viewed\
        from\
        (\
        select UserID,ProductID as Most_Viewed_product_15_Days,count(ProductID) as occurs,VisitDateTime\
            from visitorlogsdata\
            where VisitDateTime>@current_date-interval 15 day\
            and UserID is not null\
            group by UserID,ProductID\
            order by UserID,occurs desc,VisitDateTime desc\
        ) as occur_table\
        group by UserID\
        ;'

# executing query
Most_Viewed_product_15_Days = pd.read_sql_query(query, con)
Most_Viewed_product_15_Days


,UserID,Most_Viewed_product_15_Days,products_viewed
0,U100002,pr100258,4
1,U100003,Pr100079,2
2,U100004,Pr100355,4
3,U100005,Pr100234,2
4,U100006,Pr101111,1
...,...,...,...
25880,U129368,pr100071,2
25881,U129369,Pr100507,2
25882,U129370,Pr100739,1
25883,U129371,pr100051,4


In [41]:
# updating results
results['Most_Viewed_product_15_Days'] = pd.merge(results, Most_Viewed_product_15_Days,
                                                  how='left', on='UserID')['Most_Viewed_product_15_Days']
results

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days
0,U100002,NaN,2.0,52,pr100258
1,U100003,1.0,2.0,1020,Pr100079
2,U100004,1.0,15.0,340,Pr100355
3,U100005,1.0,4.0,680,Pr100234
4,U100006,1.0,1.0,54,Pr101111
...,...,...,...,...,...
34045,U136960,NaN,NaN,754,NaN
34046,U136961,NaN,NaN,59,NaN
34047,U136963,NaN,NaN,1419,NaN
34048,U136964,NaN,NaN,494,NaN


In [42]:
# filling NA values with Product101
results['Most_Viewed_product_15_Days'].fillna('Product101', inplace=True)
results


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days
0,U100002,NaN,2.0,52,pr100258
1,U100003,1.0,2.0,1020,Pr100079
2,U100004,1.0,15.0,340,Pr100355
3,U100005,1.0,4.0,680,Pr100234
4,U100006,1.0,1.0,54,Pr101111
...,...,...,...,...,...
34045,U136960,NaN,NaN,754,Product101
34046,U136961,NaN,NaN,59,Product101
34047,U136963,NaN,NaN,1419,Product101
34048,U136964,NaN,NaN,494,Product101


In [43]:
# 6) Most_Active_OS

query = 'select UserID,OS as Most_Active_OS\
        from\
        (\
        select UserID,OS,count(OS) as occur\
        from visitorlogsdata\
        where UserID is not null\
        group by UserID,OS\
        order by UserID,occur desc\
        ) as occur_table\
        group by UserID;\
        '

#executing query
Most_Active_OS = pd.read_sql_query(query, con)
Most_Active_OS


,UserID,Most_Active_OS
0,U100002,Android
1,U100003,Windows
2,U100004,Windows
3,U100005,Android
4,U100006,Android
...,...,...
34045,U136960,Windows
34046,U136961,Android
34047,U136963,Android
34048,U136964,windows


In [44]:
# updating results
results['Most_Active_OS'] = pd.merge(results, Most_Active_OS,
                                     how='left', on='UserID')['Most_Active_OS']
results

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS
0,U100002,NaN,2.0,52,pr100258,Android
1,U100003,1.0,2.0,1020,Pr100079,Windows
2,U100004,1.0,15.0,340,Pr100355,Windows
3,U100005,1.0,4.0,680,Pr100234,Android
4,U100006,1.0,1.0,54,Pr101111,Android
...,...,...,...,...,...,...
34045,U136960,NaN,NaN,754,Product101,Windows
34046,U136961,NaN,NaN,59,Product101,Android
34047,U136963,NaN,NaN,1419,Product101,Android
34048,U136964,NaN,NaN,494,Product101,windows


In [45]:
# 7) Recently_Viewed_Product

query = 'select UserID,ProductID as Recently_Viewed_Product,max(VisitDateTime)\
    from visitorlogsdata\
    where UserID is not null\
    group by UserID\
    order by UserID\
    ;\
    '
# executing query
Recently_Viewed_Product = pd.read_sql_query(query, con)
Recently_Viewed_Product


,UserID,Recently_Viewed_Product,max(VisitDateTime)
0,U100002,pr100258,2018-05-18 10:43:57
1,U100003,None,2018-05-25 02:17:49
2,U100004,pr100142,2018-05-21 13:36:01
3,U100005,Pr101814,2018-05-26 01:06:32
4,U100006,Pr101111,2018-05-21 07:21:59
...,...,...,...
34045,U136960,Pr100172,2018-05-07 00:18:20
34046,U136961,Pr101381,2018-05-07 00:12:09
34047,U136963,pr100102,2018-05-07 00:01:32
34048,U136964,Pr100769,2018-05-07 00:00:31


In [46]:
# updating results
results['Recently_Viewed_Product'] = pd.merge(results, Recently_Viewed_Product,
                                              how='left', on='UserID')['Recently_Viewed_Product']
results


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product
0,U100002,NaN,2.0,52,pr100258,Android,pr100258
1,U100003,1.0,2.0,1020,Pr100079,Windows,None
2,U100004,1.0,15.0,340,Pr100355,Windows,pr100142
3,U100005,1.0,4.0,680,Pr100234,Android,Pr101814
4,U100006,1.0,1.0,54,Pr101111,Android,Pr101111
...,...,...,...,...,...,...,...
34045,U136960,NaN,NaN,754,Product101,Windows,Pr100172
34046,U136961,NaN,NaN,59,Product101,Android,Pr101381
34047,U136963,NaN,NaN,1419,Product101,Android,pr100102
34048,U136964,NaN,NaN,494,Product101,windows,Pr100769


In [47]:
# filling NA values with Product101
results['Recently_Viewed_Product'].fillna('Product101', inplace=True)
results

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product
0,U100002,NaN,2.0,52,pr100258,Android,pr100258
1,U100003,1.0,2.0,1020,Pr100079,Windows,Product101
2,U100004,1.0,15.0,340,Pr100355,Windows,pr100142
3,U100005,1.0,4.0,680,Pr100234,Android,Pr101814
4,U100006,1.0,1.0,54,Pr101111,Android,Pr101111
...,...,...,...,...,...,...,...
34045,U136960,NaN,NaN,754,Product101,Windows,Pr100172
34046,U136961,NaN,NaN,59,Product101,Android,Pr101381
34047,U136963,NaN,NaN,1419,Product101,Android,pr100102
34048,U136964,NaN,NaN,494,Product101,windows,Pr100769


In [48]:
# 8) Pageloads_last_7_days

query = "select UserID,\
    count(case when upper(activity)='PAGELOAD' then UserID else null end) as Pageloads_last_7_days\
    from visitorlogsdata\
    where VisitDateTime>@current_date-interval 7 day\
    and UserID is not null\
    group by UserID\
    order by userID;\
    "

# executing query
Pageloads_last_7_days = pd.read_sql_query(query, con)
Pageloads_last_7_days


,UserID,Pageloads_last_7_days
0,U100003,1
1,U100004,1
2,U100005,1
3,U100006,1
4,U100008,25
...,...,...
16167,U119707,2
16168,U119708,1
16169,U119709,1
16170,U119711,2


In [49]:
# updating results
results['Pageloads_last_7_days'] = pd.merge(results, Pageloads_last_7_days,
                                            how='left', on='UserID')['Pageloads_last_7_days']
results


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days
0,U100002,NaN,2.0,52,pr100258,Android,pr100258,NaN
1,U100003,1.0,2.0,1020,Pr100079,Windows,Product101,1.0
2,U100004,1.0,15.0,340,Pr100355,Windows,pr100142,1.0
3,U100005,1.0,4.0,680,Pr100234,Android,Pr101814,1.0
4,U100006,1.0,1.0,54,Pr101111,Android,Pr101111,1.0
...,...,...,...,...,...,...,...,...
34045,U136960,NaN,NaN,754,Product101,Windows,Pr100172,NaN
34046,U136961,NaN,NaN,59,Product101,Android,Pr101381,NaN
34047,U136963,NaN,NaN,1419,Product101,Android,pr100102,NaN
34048,U136964,NaN,NaN,494,Product101,windows,Pr100769,NaN


In [50]:
# 9) Clicks_last_7_days

query = "select UserID,\
    count(case when upper(activity)='CLICK' then UserID else null end) as Clicks_last_7_days\
    from visitorlogsdata\
    where VisitDateTime>@current_date-interval 7 day\
    and UserID is not null\
    group by UserID\
    order by userID;\
    "
# executing query
Clicks_last_7_days = pd.read_sql_query(query, con)
Clicks_last_7_days


,UserID,Clicks_last_7_days
0,U100003,2
1,U100004,0
2,U100005,0
3,U100006,0
4,U100008,28
...,...,...
16167,U119707,9
16168,U119708,0
16169,U119709,0
16170,U119711,0


In [51]:
# updating results
results['Clicks_last_7_days'] = pd.merge(results, Clicks_last_7_days,
                                         how='left', on='UserID')['Clicks_last_7_days']
results


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U100002,NaN,2.0,52,pr100258,Android,pr100258,NaN,NaN
1,U100003,1.0,2.0,1020,Pr100079,Windows,Product101,1.0,2.0
2,U100004,1.0,15.0,340,Pr100355,Windows,pr100142,1.0,0.0
3,U100005,1.0,4.0,680,Pr100234,Android,Pr101814,1.0,0.0
4,U100006,1.0,1.0,54,Pr101111,Android,Pr101111,1.0,0.0
...,...,...,...,...,...,...,...,...,...
34045,U136960,NaN,NaN,754,Product101,Windows,Pr100172,NaN,NaN
34046,U136961,NaN,NaN,59,Product101,Android,Pr101381,NaN,NaN
34047,U136963,NaN,NaN,1419,Product101,Android,pr100102,NaN,NaN
34048,U136964,NaN,NaN,494,Product101,windows,Pr100769,NaN,NaN


In [52]:
# exporting results as csv
results.to_csv('results_final.csv', index=False)

 this result will be sent to Data scientist for pattern recognition

# Fin